In [3]:
using Revise
using RadiativeTransfer
using RadiativeTransfer.PhaseFunction
using RadiativeTransfer.RTM
using Distributions

┌ Info: Precompiling RadiativeTransfer [ace8185b-fa20-42f5-912d-1e850dbe6a09]
└ @ Base loading.jl:1278


In [4]:
"Generate aerosol optical properties"

# Wavelength (just one for now)
λ = 0.55       # Incident wavelength
depol = 0.0
# Truncation 
Ltrunc = 20             # Truncation  
truncation_type   = δBGE(Ltrunc, 2.0)

# polarization_type
polarization_type = Stokes_IQUV()

# Quadrature points for RTM
Nquad, qp_μ, wt_μ = rt_set_streams(RTM.RadauQuad(), Ltrunc, 60.0, [0.0, 15.0, 30., 45., 60.])

# Aerosol particle distribution and properties
μ            = [0.3,2.0]       # Log mean radius
σ            = [2.0,1.8]       # Log stddev of radius
r_max        = [30.0,30.0]     # Maximum radius
nquad_radius = [2500,2500]     # Number of quadrature points for integrating of size dist.
nᵣ           = [1.3, 1.66]     # Real part of refractive index
nᵢ           = [0.001,0.0003]  # Imag part of refractive index

#Aerosol vertical distribution profiles
p₀           = [50000., 20000.] # Pressure peak [Pa]
σp          = [5000., 2000.]   # Pressure peak width [Pa]

size_distribution = [LogNormal(log(μ[1]), log(σ[1])), LogNormal(log(μ[2]), log(σ[2]))]

# Create the aerosols (needs to be generalized through loops):
aero1 = make_univariate_aerosol(size_distribution[1], r_max[1], nquad_radius[1], nᵣ[1], nᵢ[1])
aero2 = make_univariate_aerosol(size_distribution[2], r_max[2], nquad_radius[2], nᵣ[2], nᵢ[2])

# Define some details, run aerosol optics
model_NAI2_aero1 = make_mie_model(NAI2(), aero1, λ, polarization_type, truncation_type)
aerosol_optics_NAI2_aero1 = compute_aerosol_optical_properties(model_NAI2_aero1);
# Truncate:
aerosol_optics_trunc_aero1 = PhaseFunction.truncate_phase(truncation_type, aerosol_optics_NAI2_aero1)

# Define some details, run aerosol optics
model_NAI2_aero2 = make_mie_model(NAI2(), aero2, λ, polarization_type, truncation_type)
aerosol_optics_NAI2_aero2 = compute_aerosol_optical_properties(model_NAI2_aero2);
# Truncate:
aerosol_optics_trunc_aero2 = PhaseFunction.truncate_phase(truncation_type, aerosol_optics_NAI2_aero2)

# Rayleigh Greek
GreekRayleigh = PhaseFunction.get_greek_rayleigh(depol)


┌ Info: Fraction of size distribution cut by max radius: 1.5279000287193867e-9 %
└ @ RadiativeTransfer.PhaseFunction /Users/sanghavi/GDrive/code/github/RadiativeTransfer.jl/src/PhaseFunction/mie_helper_functions.jl:263
Computing PhaseFunctions Siewert NAI-2 style ...100%|███| Time: 0:00:02
┌ Info: Fraction of size distribution cut by max radius: 0.00020406457610366857 %
└ @ RadiativeTransfer.PhaseFunction /Users/sanghavi/GDrive/code/github/RadiativeTransfer.jl/src/PhaseFunction/mie_helper_functions.jl:263
Computing PhaseFunctions Siewert NAI-2 style ...100%|███| Time: 0:00:01


GreekCoefs{Float64}([0.0, 0.0, 3.0], [1.0, 0.0, 0.5], [0.0, 0.0, 1.224744871391589], [0.0, 1.5, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0])

In [5]:
" Atmospheric Profiles, basics, needs to be refactore entirely"
file = "/Users/sanghavi/GDrive/code/github/atm_profiles/MERRA300.prod.assim.inst6_3d_ana_Nv.20150613.hdf.nc4"   
timeIndex = 2 # There is 00, 06, 12 and 18 in UTC, i.e. 6 hourly data stacked together

# What latitude do we want? 
myLat = 34.1377;
myLon = -118.1253;

# Read profile (and generate dry/wet VCDs per layer)
profile_caltech = RTM.read_atmos_profile(file, myLat, myLon, timeIndex);

# Compute layer optical thickness for Rayleigh (surface pressure in hPa) 
τRayl =  RTM.getRayleighLayerOptProp(profile_caltech.psurf/100, λ, depol, profile_caltech.vcd_dry);
ϖRayl = ones(length(τRayl))

#Compute Naer aerosol optical thickness profiles
τAer_1 = RTM.getAerosolLayerOptProp(1.0, p₀[1], σp[1], profile_caltech.p_levels)
τAer_2 = RTM.getAerosolLayerOptProp(0.3, p₀[2], σp[2], profile_caltech.p_levels)

# Can be done with arbitrary length later:
τAer = [τAer_1 τAer_2]
@show sum(τAer_1), sum(τAer_2)
ϖAer = [aerosol_optics_NAI2_aero1.ω̃ aerosol_optics_NAI2_aero2.ω̃];
fᵗ   = [aerosol_optics_trunc_aero1.fᵗ aerosol_optics_trunc_aero2.fᵗ];


(sum(τAer_1), sum(τAer_2)) = (1.0, 0.30000000000000004)


In [8]:
@show size([τAer_1 τAer_2])
@show ϖAer
@show ϖRayl

size([τAer_1 τAer_2]) = (73, 2)
ϖAer = [0.9822643453151417 0.9693947188593625]
ϖRayl = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


72-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [9]:
m = 0
RaylZ⁺⁺, RaylZ⁻⁺     = PhaseFunction.compute_Z_moments(polarization_type, qp_μ, GreekRayleigh, m);
aero1_Z⁺⁺, aero1_Z⁻⁺ = PhaseFunction.compute_Z_moments(polarization_type, qp_μ, aerosol_optics_trunc_aero1.greek_coefs, m);
aero2_Z⁺⁺, aero2_Z⁻⁺ = PhaseFunction.compute_Z_moments(polarization_type, qp_μ, aerosol_optics_trunc_aero2.greek_coefs, m);

Aer𝐙⁺⁺ = [aero1_Z⁺⁺, aero2_Z⁺⁺];
Aer𝐙⁻⁺ = [aero1_Z⁻⁺, aero2_Z⁻⁺];

iz = 10
τ, ϖ, Z⁺⁺, Z⁻⁺  = RTM.construct_atm_layer(τRayl[iz], τAer[iz,:], ϖRayl[iz], ϖAer, fᵗ, RaylZ⁺⁺, RaylZ⁻⁺, Aer𝐙⁺⁺, Aer𝐙⁻⁺)
@show τ, ϖ
@show τAer[iz,:], τRayl[iz]

(τ, ϖ) = (8.309208591510007e-6, 1.0)
(τAer[iz, :], τRayl[iz]) = ([1.0921404610080213e-25, 9.601082758390844e-26], 8.309208591510007e-6)


([1.0921404610080213e-25, 9.601082758390844e-26], 8.309208591510007e-6)

In [16]:
RTM.doubling_number(0.003, 0.3)

(0.0023437499999999986, 7)